In [1]:
import csv
import ipywidgets as widgets

In [3]:
def generate_csv(dict, output_path, append=True): 
    """    
    generates a csv to output_path to organize sequencing prep based on dict. dict should come from generate_dict. 
    returns nx2 matrix of indexes and final files names
    """
    
    # TODO: maybe use pandas for csv writing to simplify code
    
    header = dict['header']
    matrix = []
    index = 1
    for i in range(len(dict['genes'])): 
        for j in range(len(dict['clones'])): 
            for k in range(len(dict['clones'][j])): 
                matrix.append([dict['base'], index, dict['genes'][i], dict['clones'][j][k], dict['oligos'][j][k]])
                index += 1
    edit = "a" if append else "w"
    csv_file = open(output_path, edit)
    csvwriter = csv.writer(csv_file)
    if not append: 
        csvwriter.writerow(header)
    csvwriter.writerows(matrix)
    #csvwriter.writerows(generate_names(matrix))
    csv_file.close()
    
    return generate_names(matrix)

            

In [4]:
def generate_dict(base, genes, clones, oligos, oligos_first=False, same_last=False): 
    """
    generates dict to pass into generate_csv()
    
    examples/info for inputs: 
        base: string
        genes: [gene1, gene2...]
        clones: [[1-3], [4-6]...] (must match genes, may want to automate this somehow)
        oligos: [[2, 3, 4], [2, 6, 7]...] (dimensions must match clones)
        clones_first: if true, clones higher in heirarchy than oligos. 
        same_oligos: if true, oligos can just be [24, 5, 3] and assume applies to all sets of clones
    """
    result = {};
    if not oligos_first: 
        result['header'] = ['base', 'index', 'gene', 'clone', 'oligo']
        if not same_last: 
            result['clones'] = clones
            result['oligos'] = oligos
        else: 
            result['clones'], result['oligos'] = [], []
            for i in range(len(clones)):
                temp_clones, temp_oligos = [], []
                for j in range(len(oligos)): 
                    temp_clones.append(clones[i])
                    temp_oligos.append(oligos[j])
                result['clones'].append(temp_clones)
                result['oligos'].append(temp_oligos)
                    
    else: 
        result['header'] = ['base', 'index', 'gene', 'oligo', 'clone']
        if not same_last: 
            result['oligos'] = clones
            result['clones'] = oligos
        else: 
            result['oligos'], result['clones'] = [], []
            for i in range(len(oligos)):
                temp_clones, temp_oligos = [], []
                for j in range(len(clones)): 
                    temp_clones.append(oligos[i])
                    temp_oligos.append(clones[j])
                result['clones'].append(temp_clones)
                result['oligos'].append(temp_oligos)
    
    result['base'] = base
    result['genes'] = genes
    
    
    
    return result

In [5]:
def generate_names(matrix): 
    """
    returns a nx2 matrix of indexes + names from matrix of name elements
    """
    result = []
    for row in matrix: 
        name = ""
        index = 0
        for col in row:
            if index == 1: 
                index += 1
                continue
            if index == len(row)-1: 
                name+='_seq_'
            name += str(col)
            index += 1
        result.append([row[1], str(row[1])+'_'+name])
    return result
        

In [8]:
widgets.interact_manual(generate_dict, base="", genes="", clones="", oligos="")

interactive(children=(Text(value='', description='base'), Text(value='', description='genes'), Text(value='', …

<function __main__.generate_dict(base, genes, clones, oligos, oligos_first=False, same_last=False)>

In [10]:

dict = generate_dict('20220303', 
                     ['arpc3-mSI_test_new_pols_'],
                     ['wild_type_gDNA_amp_214_221', 'amp_2_3'],
                      
                     [214, 221, 99, 220, 218],
                     oligos_first=False,
                     same_last=True)
generate_csv(dict, "Test Data/name_ref.csv", append=False)

[[1, '1_20220303arpc3-mSI_test_new_pols_wild_type_gDNA_amp_214_221_seq_214'],
 [2, '2_20220303arpc3-mSI_test_new_pols_wild_type_gDNA_amp_214_221_seq_221'],
 [3, '3_20220303arpc3-mSI_test_new_pols_wild_type_gDNA_amp_214_221_seq_99'],
 [4, '4_20220303arpc3-mSI_test_new_pols_wild_type_gDNA_amp_214_221_seq_220'],
 [5, '5_20220303arpc3-mSI_test_new_pols_wild_type_gDNA_amp_214_221_seq_218'],
 [6, '6_20220303arpc3-mSI_test_new_pols_amp_2_3_seq_214'],
 [7, '7_20220303arpc3-mSI_test_new_pols_amp_2_3_seq_221'],
 [8, '8_20220303arpc3-mSI_test_new_pols_amp_2_3_seq_99'],
 [9, '9_20220303arpc3-mSI_test_new_pols_amp_2_3_seq_220'],
 [10, '10_20220303arpc3-mSI_test_new_pols_amp_2_3_seq_218']]